In [1]:
import pandas as pd

In [2]:
ins = pd.read_csv('InstallmentsTable.csv')
ins.set_index(['borrower_id', 'loan_id'], inplace=True)

## Check Which Loans Have Defaulted  (Miss 2 installments In a row)

In [3]:
#Mask for maximum lag
lag_mask = ins.groupby(level=1).shift(1).due_date.isnull()
#Mask for defaults (must occur beyond lag window if at beginning of installment series)
default_mask = (#ins.groupby(level=1).shift(2).paid_amount.isnull() & 
                ins.groupby(level=1).shift(1).paid_amount.isnull() & 
                ins.paid_amount.isnull() & ~lag_mask)

## Calculate Default Date

In [4]:
defaults = ins[default_mask].groupby(level = ['borrower_id', 'loan_id']).agg({'due_date':'min'})
defaults.columns = ['default_date']

In [5]:
targets = (ins.
join(defaults[['default_date']]).
groupby(level = ['borrower_id', 'loan_id']).
agg({'default_date':{'default_date': 'min', 'default_flag':lambda x: max(~x.isnull())}}))

targets.columns = targets.columns.droplevel()

targets.head(10)

default_flag         default_date
borrower_id loan_id                                  
129         1              False                  NaN
            58              True  2012-07-04 10:38:55
130         2              False                  NaN
            59              True  2011-02-22 07:57:19
131         3              False                  NaN
            60              True  2011-02-22 07:58:00
146         4              False                  NaN
            69             False                  NaN
            408            False                  NaN
            4393           False                  NaN

In [6]:
targets.to_csv('defaults.csv')

In [8]:
#check calculation
#ins[ins.index.get_level_values('borrower_id') == 131][['due_date', 'paid_amount']]